### Project Kojak

** The problem  **

We will attempt to identify amboguously defined words - words that are homographs (spelled the same, but with multiple meanings) and determine the exact meaning of the word from a context window.

Here we attempt to do this in a few stages
1. train a word embedding on some training corpus using skip-gram (Here we use 1000 sholarly research papers) 
2. identify common homographs and extract the various context windows
3. interpret the context windows as vectors in the embedding space and appy a clustering algorith (DBSCAN). Each cluster is interpreted as a distinct definition of the homograph. Each cluster then is representative vector.
4. apply to a test corpus - match context of given homograph to most similar group.


### This notebook

Loads a pre-trained word embedding model, uses DBSCAN clustering to identify several 'definitions' of a set of homographs, and saves those definitions for later use.

In [1]:
import gensim
import json
import os
import re
import time
from nltk.corpus import stopwords
from nltk import tokenize
from pprint import pprint



In [2]:
# Declare stopwords, preprocess the data from source file

stop = stopwords.words('english')
stop += ['?','!',':',';','[',']','[]','“' ]
stop += ['.', ',', '(', ')', "'", '"',"''",'""',"``",'”', '“', '?', '!', '’', 'et', 'al', 'al.']
stop = set(stop)

class MyPapers(object):
    # a memory-friendly way to load a large corpora
     def __init__(self, dirname):
            self.dirname = dirname
 
     def __iter__(self):
        with open(self.dirname) as data_file:    
            data = json.load(data_file)
        # iterate through all file names in our directory
        for paper in data:
            sentences = tokenize.sent_tokenize(paper)
            for line in sentences:
                try:
                    line = [word for word in paper['full_text'].lower().split() if word not in stop]
                    line = [re.sub(r'[?\.,!:;\(\)“\[\]]',' ',l) for l in line]
                    yield line
                except:
                    print("Empty line found")
                    continue
                

## Word embeddings

Import word2vec word embeddings trained on 2848 scholarly journal articles

In [3]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from functools import reduce

In [4]:
model = gensim.models.word2vec.Word2Vec.load("data/journal.txt")

In [5]:
model.corpus_count

2848

In [6]:
vectors = model.wv

In [7]:
len(vectors.vocab)

207432

** contexts to vectors **

In [8]:
from collections import Counter, defaultdict
# The function takes as arguments a list of tokenized documents and a window size
# and returns each word in the document along with its window context as a tuple

def generate_word_counts(documents, window_size = 6):
    maxlen = window_size*2
    counts = Counter()
    
    for document in documents:
        L = len(document)
        # Choose the target word
        for index, word in enumerate(document):
            # Create the window
            s = index-window_size
            e = index+window_size+1
                    
            in_words = []
            context_words = []
            # Create the input/outputs for skipgrams
            for i in range(s, e):
                if i != index and 0 <= i < L:
                    #in_words.append([word])
                    context_words.append(document[i])
            x = word
            y = context_words
            counts[x] += 1
            for _ in y:
                counts[_] += 1

    return counts

# Takes list of word tokens as arguments
# Returns a list of vectors whose components are the arithmetic mean of the 
# corresponding component of all of the input vectors

def get_vectors(word_list):
    vecs = []
    for word in word_list:
        vecs.append(vectors[word])        
    return vecs

# Takes list of vectors as arguments
# Returns a single vector whose components are the arithmetic mean of the 
# corresponding component of all of the input vectors

def vector_average(vector_list):
    A = np.array(vector_list)
    dim = A.shape[0]
    ones = np.ones(dim)
    return ones.dot(A)/dim

# Takes list of tokenized documents, target word and window size as arguments
# Returns list of vectors where each vector represents the context window 
# of the target word in the word embedding space

def context2vectors(documents,target,window_size = 6):

    context_vectors = []

    for document in documents:
        if target in document:
            windows = generate_windows([document],window_size)
            for w in windows:
                if w[0] == target:
                    context_vectors.append(vector_average2(get_vectors(w[1])))
                    
    return context_vectors
# Takes list of vectors as arguments
# Returns a single vector whose components are the arithmetic mean of the 
# corresponding component of all of the input vectors weighted by Inverse Document Frecuency

def vector_average2(words): #, word_counts, vectors):
    total = sum(list(word_counts.values()))
    words = [x for x in words if x in list(vectors.vocab.keys())]
    vector_list = list(map((lambda x: vectors[x]*np.log((1 + total)/(1 + word_counts[x]))),words))
    
    if len(vector_list) == 0:
        return 0
    elif len(vector_list) == 1:
        vector_sum = vector_list[0]
    else:
        vector_sum = reduce((lambda x,y: np.add(x,y)),vector_list)
        
    weighted_average = (1.0/len(words))*vector_sum
    
    return weighted_average

# Takes list of tokenized documents, target word and window size as arguments
# Returns list of vectors where each vector represents the context window 
# of the target word in the word embedding space

def context2vectors2(documents,target,window_size = 6):

    context_vectors = []

    for document in documents:
        if target in document:
            windows = generate_windows([document],window_size)
            for w in windows:
                if w[0] == target:
                    context_vectors.append(vector_average2(w[1]))
                    
    return context_vectors

In [9]:
#Instantiate iterable on the data

#papers is an iterable of scholarly papers, tokenized for prcessing
papers = MyPapers('data/train_data.json') 

In [10]:
def MyPapers_plus(papers):
    
    phrases = gensim.models.phrases.Phrases(sentences = papers, min_count = 5, threshold = 150)
    bigram = gensim.models.phrases.Phraser(phrases)
    phrases2 = gensim.models.phrases.Phrases(sentences = bigram[papers], min_count = 5, threshold = 300)
    trigram = gensim.models.phrases.Phraser(phrases2)
    
    return trigram[bigram[papers]]

In [11]:
word_counts = generate_word_counts(MyPapers_plus(papers))

In [12]:
word_counts['new_york_city']

143

In [13]:
dictionary = gensim.corpora.dictionary.Dictionary(MyPapers_plus(papers))
text = [dictionary.doc2bow(c) for c in MyPapers_plus(papers)]

** Clustering with DBSAN **

Use DBSCAN to determine similar usages of the target homographs. Each of these similar usages will be combined to a representative vector in the embedding space and constitute a "definition" of that word.

In [14]:
# The function takes as arguments a list of tokenized documents and a window size
# and returns each word in the document along with its window context as a tuple

def generate_windows(documents, window_size):
    maxlen = window_size*2
    
    for document in documents:
        L = len(document)
        # Choose the target word
        for index, word in enumerate(document):
            # Create the window
            s = index-window_size
            e = index+window_size+1
                    
            in_words = []
            context_words = []
            # Create the input/outputs for skipgrams
            for i in range(s, e):
                if i != index and 0 <= i < L:
                    #in_words.append([word])
                    context_words.append(document[i])
            x = word
            y = context_words

            yield(x,y)
            

#Arguments: The desired cluster number, a list of documents making up the corpus, the target homograph
#           a list of labels for the conext sentences indicating the homographs usage, and window size
# The function prints the representative context windows for the target word within the desired cluster   

def print_cluster_context(cluster_number, documents, target, labels, window_size = 6):
    
    context_vectors = []

    for document in documents:
        text = document
        if target in text:
            #print(target)
            windows = generate_windows([text],window_size)
            #print windows[:2]
            for w in windows:
                if w[0] == target:
                    context_vectors.append((w[1]))
    for i, label in enumerate(labels):
        if label == cluster_number:
            print(context_vectors[i])
            
#Arguments: The desired cluster number, a list of documents making up the corpus, the target homograph
#           a list of labels for the conext sentences indicating the homographs usage, and window size
# The function prints the representative context windows for the target word within the desired cluster   

def cluster_context(documents, target, labels, window_size = 6):
    
    context_windows = []

    for document in documents:
        text = document
        if target in text:
            #print(target)
            windows = generate_windows([text],window_size)
            #print windows[:2]
            for w in windows:
                if w[0] == target:
                    context_windows.append((w[1]))
                    
    cluster_windows = defaultdict(list)                
    for i, label in enumerate(labels):
        cluster_windows[label].append(context_windows[i])
    
    return cluster_windows

# Arguments: List of vectors, each representing a context window and a list of labels 
#            corresponding to the context vectors
# Returns:   A dictionary where keys are the identified labels from clustering and the value is a single 
#            representing the cluster

def identify_definition(context_vectors, labels):
    
    cluster_numbers = set(labels)
    definitions = dict()
    cluster_vectors = defaultdict(list)
    if len(set(labels)) == 1:
        print("No consistent definition found")
        definitions[0] = np.zeros(len(context_vectors[0]))
        return definitions
    
    for i, label in enumerate(labels):
        cluster_vectors[label].append(context_vectors[i])
    
    for key in cluster_vectors.keys():
        if key < 0:
            continue
        else:
            v = vector_average(cluster_vectors[key])
            definitions[key] = v/np.linalg.norm(v)
                    
    return definitions


In [51]:
target = u'train'
context_vectors = context2vectors2(MyPapers_plus(papers), target)

In [55]:
epsilon = .12

In [52]:
dbscan = DBSCAN(eps = epsilon, metric = 'cosine', algorithm = 'brute', min_samples = 5)
dbscan.fit(context_vectors)

DBSCAN(algorithm='brute', eps=0.12, leaf_size=30, metric='cosine',
    min_samples=5, n_jobs=1, p=None)

In [53]:
labels = dbscan.labels_
n_clusters = len(set(labels)) # - (1 if -1 in labels else 0)
print(n_clusters)

3


In [54]:
labels

array([ 1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1,  1,  0, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  0,  0,  0,
        0, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  1, -1])

In [82]:
target_definitions = identify_definition(context_vectors, labels)

In [83]:
contexts = cluster_context(MyPapers_plus(papers), target, labels)

# Recording Definitions

Use the DBSCAN clusters to determine the various definitions of a word, then create a dictionary for the word

In [15]:
def define_window(window, dictionary):
    cosine_dists = []
    wv = vector_average2(window)
    window_vector = wv/np.linalg.norm(wv)
    for k,v in dictionary.items():
        cosine_dists.append((1 - np.dot(window_vector, v),k))
    cosine_dists.sort()
    print(cosine_dists)
    
    return cosine_dists[0][1]
    
def extract_dictionary(papers, homographs):
    dictionary = dict()
    for word in homographs:
        print("Calculating context vectors for {}".format(word))
        context_vectors = context2vectors2(papers, word)
        print("Clustering...")
        dbscan = DBSCAN(eps = epsilon, metric = 'cosine', algorithm = 'brute', min_samples = 5)
        dbscan.fit(context_vectors)
        labels = dbscan.labels_
        print("found {} distinct definitions".format(len(set(labels))))
        print("Building definition for {}".format(word))
        dictionary[word] = identify_definition(context_vectors, labels)
        
    print("Dictionary complete")    
    return dictionary    

In [33]:
homographs = ['attribute', 'bank', 'charge', 'second', 'train']

dictionary = extract_dictionary(papers, homographs)

Calculating context vectors for attribute
Clustering...
Calculating context vectors for bank
Clustering...
Calculating context vectors for charge
Clustering...
Calculating context vectors for feet
Clustering...
Calculating context vectors for second
Clustering...
Calculating context vectors for train
Clustering...


In [42]:
dictionary['bank'].keys()

dict_keys([0, 1])

## Testing

In [57]:
# TEST WINDOW #

target = u'train'
context_vectors = context2vectors2(MyPapers_plus(papers), target)
dbscan = DBSCAN(eps = epsilon, metric = 'cosine', algorithm = 'brute', min_samples = 4)
dbscan.fit(context_vectors)
labels = dbscan.labels_
target_definitions = identify_definition(context_vectors, labels)
#target_definitions = dictionary[target]


contexts = cluster_context(MyPapers_plus(papers), target, labels)
correct = 0
wrong = 0
for c in contexts:
    for window in contexts[c]:
        d = define_window(window, target_definitions)
        if c == d:
            correct += 1
        else:
            print("Cluster {}, defined as {} \n".format(c, d), window, '\n')
            if c != -1:
                wrong += 1
                
print("Number correct: {}\nNumber wrong: {}".format(correct, wrong))

[(0.07683784553169537, 1), (0.22122995662929146, 0)]
[(0.064747383698259942, 1), (0.31271402436150497, 0)]
[(0.070376832753871299, 1), (0.28017731437017335, 0)]
[(0.05249510644251687, 1), (0.21839005681020573, 0)]
[(0.060763873207947849, 1), (0.26094942894628526, 0)]
[(0.032630030628247209, 1), (0.21364742175876938, 0)]
[(0.14818864957332134, 0), (0.3066886185737927, 1)]
Cluster -1, defined as 0 
 ['obtained', 'means', 'local', 'field', 'potential', 'spike', 'analysis ', 'wide_range', 'topological', 'network', 'measures', ' 14'] 

[(0.13866049494623311, 0), (0.29864189469882385, 1)]
Cluster -1, defined as 0 
 ['putting', 'probes', 'local', 'field', 'potential', 'spike', 'analysis ', 'aim', 'reconstruct', 'functional', 'time-varying', 'brain'] 

[(0.18461941947731031, 1), (0.27436335460479133, 0)]
Cluster -1, defined as 1 
 ['improving', 'access', 'psychological', 'therapies', ' iapt ', 'developed', 'low', 'intensity', 'psychological', 'wellbeing', 'practitioners', ' pwps '] 

[(0.16149


[(0.13322873440308158, 0), (0.30959955519254356, 1)]
Cluster -1, defined as 0 
 ['similarities', 'uses', 'kernel', 'drug–target_pairs', 'known', 'labels', 'svm-classifier ', 'approaches', 'presented', ' – ', 'represent', 'drug–target'] 

[(0.1161721657550332, 0), (0.24728606287478516, 1)]
Cluster -1, defined as 0 
 ['simplicity ', 'general', 'recommendation', 'efficiently', 'defining_ad', 'would', 'powerful', 'classifier', 'use', 'built-in', 'class', 'probability'] 

[(0.12059867808953206, 1), (0.29381687699642811, 0)]
Cluster -1, defined as 1 
 ['started', 'change', '2005 ', 'government', 'launched', 'program', 'professors', 'identifying', 'gifted', 'students ', '2010 ', 'example '] 

[(0.12192031457530117, 0), (0.24772056067143344, 1)]
Cluster -1, defined as 0 
 ['recognition ', 'authors', 'divide', 'input', 'image', 'patches', 'convolutional_neural', 'network', ' cnn ', 'patch', 'fashion', 'find'] 

[(0.11181050739496978, 0), (0.28072215698793523, 1)]
Cluster -1, defined as 0 
 ['p

[(0.17383358732182752, 1), (0.20810594065382038, 0)]
Cluster -1, defined as 1 
 ['learners', 'prepare', 'conventional', 'e-learning', 'material ', 'also', 'aspects', 'image', 'acquisition ', 'unique', 'haptic', 'experience '] 

[(0.14561275115441918, 0), (0.23537551605605733, 1)]
Cluster -1, defined as 0 
 ['light', 'railway ', 'connections ', 'bus', 'station ', 'high-speed', 'station also ', 'one', 'aims', 'also', 'create', 'mixed-use'] 

[(0.11936434968356113, 0), (0.20921706416948826, 1)]
Cluster -1, defined as 0 
 ['uses', 'embodied', 'tasks ', 'folding', 'cutting', 'paper ', 'two-dimensional', 'three-dimensional', 'spatial_thinking ', 'analyses', 'explored', 'spatial_thinking'] 

[(0.18542400659298941, 1), (0.21243092638100436, 0)]
Cluster -1, defined as 1 
 ['embodied', 'spatial', 'training', 'may', 'provide', 'means', 'fundamental', 'cognitive', 'skill ', 'spatial_thinking ', 'turn', 'important'] 

[(0.19395178355139042, 1), (0.21690514870690492, 0)]
Cluster -1, defined as 1 
 [

In [61]:
from nltk import tokenize
p = "Hello, jerks! How's it going? By, the way. You smell."
tokenize.sent_tokenize(p)



['Hello, jerks!', "How's it going?", 'By, the way.', 'You smell.']